In [1]:
# import dependencies
# import pandas as pd
# import numpy as np
import random
import json


In [20]:
# select desired state
state = 'missouri'

# access json file with data to be sonified for avaliable states
stateData = json.load(open('../resources/stateData.json'))

# list to hold probabilities for which demographic a new case will belong to
infectProbs = []
# list to hold probability that an infection will result in death for each demographic
deathProbs = []

streamProbs = []

# populate lists above
for data in stateData[state]:
    infectProbs.append(stateData[state][data]['chance_of_infection'])
    deathProbs.append(stateData[state][data]['chance_of_death'])
    streamProbs.append(stateData[state][data]['inverted_ccvi'])


In [3]:
# function to determine how many cases correspond to a particular threshold
def nthThresh(n,thresh):
    return 100 * n + thresh

# function to adjust spreadProbs weights and calculate r0 for corresponding threshold
def newThresh():
    global n
    global thresh
    if n == 1:
        spreadProbs[3] -= 1
        spreadProbs[2] += 1
    elif n == 2:
        spreadProbs[3] -= 1
        spreadProbs[1] += 1
    elif n == 13:
        spreadProbs[2] -= 1
        spreadProbs[0] += 1
    elif n == 14:
        spreadProbs[1] -= 1
        spreadProbs[0] += 1
    elif n != 0 and n < 13:
        spreadProbs[3] -= 1
        spreadProbs[0] += 1
    r0 = (spreadProbs[1] + 2*spreadProbs[2] + 3*spreadProbs[3]) / sum(spreadProbs)
    # calculate new threshold
    thresh = nthThresh(n, thresh)
    # add one to threshold n value
    n += 1
    
def newTrans():
    global trans
    t = random.randint(1,12)
    if trans+t > 12:
        trans -= t
    elif trans-t < -12:
        trans += t
    else:
        trans += t*random.choice([-1,1])

# function to select rescaled incubation period as a delay in milliseconds.
def incubate(n):
    if n == 1:  
        sub = 3
        return random.randint(0, sub*40)*2000/sub
    elif n == 2:
        sub = random.choice([3,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 3:
        sub = random.choice([6,4])
        return random.randint(0, sub*40)*2000/sub
    elif n == 4:
        sub = random.choice([6,8])
        return random.randint(0, sub*40)*2000/sub
    elif n == 5:
        sub = random.choice([6,8,5])
        return random.randint(0, sub*40)*2000/sub
    else:
        sub = random.choice([6,8,5,7])
        return random.randint(0, sub*40)*2000/sub

# function to select a value based on a probability distribution array
def select(probsArray):
    return random.choices(list(range(len(probsArray))),weights=probsArray)[0]

# function to decide between one and zero based on probability that 1 will be chosen
def decide(prob):
    return random.choices([0,1], weights=[100-prob, prob])[0]

def perspective(time):
    probs = [0, 0, 0, 0, 0, 0]
    for demo in range(0, len(time), 3):
        print(time[demo])
        probs[time[demo]] += streamProbs[time[demo]]
    return select(probs)

# function to generate midi notes
def midi(trans, death):
    note = random.choice(midiNotes)
    return note + trans - death



In [4]:
# weighted probabilities for #people the current case can spread infection to
# (from left to right, values refer to weights for 0, 1, 2, and 3 new infections)
spreadProbs = [0,0,0,12]
# symptomatic cases that have occured
sympCount = 0
# deaths that have occured
deathCount = 0
# case count threshold for when new r0 shoud be calculated
thresh = 0
# nth threshold
n = 1
# infection rate
r0 = 3
# transposition in semitones
trans = 0
# cases remaining (one case to start it off)
times = []
# which case
case = 0
# array of midi notes
midiNotes = [48, 51, 55, 58, 60, 62, 63, 65, 67, 68, 70, 74, 77, 80]
# dictionary with all data
weathering = {
    'time': [],
    'demo': [],
    'note': [],
    'death': [],
    'n': [],
}


In [5]:
times.append(incubate(n))

while len(times) > 0:
    
    # determine how much time has elapse since beginning of piece
    now = min(times)
    
    # if symptomatic add case info to weathering
    if decide(40) == 0:

        # add current case to total # of cases
        # (only sympotmatic cases adjust threshold)
        sympCount += 1
        
        if sympCount % 50 == 0:
            newTrans()

         # if case count is at or over threshold number
        if sympCount > nthThresh(n, thresh):
            newThresh()
            
        weathering['time'].append(now)

        # select which demographic the current case belongs to
        demo = select(infectProbs)
        weathering['demo'].append(demo)

        # decide/record if case results in death or not
        death = decide(deathProbs[demo])
        weathering['death'].append(death)

        # select which note
        weathering['note'].append(midi(trans, death))

        weathering['n'].append(n)
    
   

    # decide how many people the current case infects and add them to more
    for i in range(select(spreadProbs)):
        
        # select incubation period (delay time)
        delay = incubate(n)
        
        # determine time elapsed since beginning and append to times list
        time = now + delay
        times.append(time)

    # remove current case from timestamps of remaining cases
    times.pop(times.index(now))


In [6]:
result = sorted(list(zip(weathering['time'], weathering['demo'], weathering['note'], weathering['death'], weathering['n'])))

len(result)


11477

In [7]:
result

[(78666.66666666666, 1, 68, 0, 1),
 (81333.33333333333, 3, 60, 0, 1),
 (99333.33333333333, 0, 60, 0, 1),
 (112000.0, 1, 48, 0, 1),
 (113333.33333333331, 5, 63, 0, 1),
 (117999.99999999999, 2, 55, 0, 1),
 (120000.0, 0, 60, 0, 1),
 (120666.66666666666, 0, 70, 0, 1),
 (122666.66666666664, 3, 63, 0, 1),
 (122666.66666666664, 5, 74, 0, 1),
 (133333.3333333333, 1, 65, 0, 1),
 (137333.3333333333, 0, 80, 0, 1),
 (138000.0, 3, 68, 0, 1),
 (138666.66666666663, 5, 63, 0, 1),
 (147999.99999999997, 5, 48, 0, 1),
 (148000.0, 0, 67, 0, 1),
 (151333.33333333334, 0, 77, 0, 1),
 (152666.66666666666, 2, 62, 0, 1),
 (158666.66666666666, 0, 48, 0, 1),
 (162000.0, 0, 62, 0, 1),
 (162666.66666666666, 1, 51, 0, 1),
 (164666.66666666666, 1, 51, 0, 1),
 (164666.66666666666, 3, 62, 0, 1),
 (165999.99999999997, 2, 77, 0, 1),
 (166666.66666666663, 5, 68, 0, 1),
 (167333.3333333333, 0, 62, 0, 1),
 (167333.3333333333, 1, 80, 0, 1),
 (169333.3333333333, 0, 48, 0, 1),
 (169333.3333333333, 5, 62, 0, 1),
 (170666.666666

In [8]:
# for i, r in enumerate(result):
#     print(f'{i}: {r[4]}')



In [9]:
meep = float(round(result[3][0],6))
moop = float(round(result[4][0],6))
print(meep)
print(moop)

moop - meep


112000.0
113333.333333


1333.3333330000023

In [10]:
timing = {}
ms = 0
infections = -1

for case in result:
    
    time = round(case[0], 6)

    if time != ms:
        infections += 1
        timing[infections] = {'demoNotes': [case[1], case[2], case[3]]}
        timing[infections]['delay'] = round(time - ms, 6)
        ms = time

    else:
        for i in range(1,4):
            timing[infections]['demoNotes'].append(case[i])


In [11]:
timing



{0: {'demoNotes': [1, 68, 0], 'delay': 78666.666667},
 1: {'demoNotes': [3, 60, 0], 'delay': 2666.666666},
 2: {'demoNotes': [0, 60, 0], 'delay': 18000.0},
 3: {'demoNotes': [1, 48, 0], 'delay': 12666.666667},
 4: {'demoNotes': [5, 63, 0], 'delay': 1333.333333},
 5: {'demoNotes': [2, 55, 0], 'delay': 4666.666667},
 6: {'demoNotes': [0, 60, 0], 'delay': 2000.0},
 7: {'demoNotes': [0, 70, 0], 'delay': 666.666667},
 8: {'demoNotes': [3, 63, 0, 5, 74, 0], 'delay': 2000.0},
 9: {'demoNotes': [1, 65, 0], 'delay': 10666.666666},
 10: {'demoNotes': [0, 80, 0], 'delay': 4000.0},
 11: {'demoNotes': [3, 68, 0], 'delay': 666.666667},
 12: {'demoNotes': [5, 63, 0], 'delay': 666.666667},
 13: {'demoNotes': [5, 48, 0, 0, 67, 0], 'delay': 9333.333333},
 14: {'demoNotes': [0, 77, 0], 'delay': 3333.333333},
 15: {'demoNotes': [2, 62, 0], 'delay': 1333.333334},
 16: {'demoNotes': [0, 48, 0], 'delay': 6000.0},
 17: {'demoNotes': [0, 62, 0], 'delay': 3333.333333},
 18: {'demoNotes': [1, 51, 0], 'delay': 66

In [12]:
with open("../resources/weathering.json", "w") as outfile:
    json.dump(timing, outfile)
    